# Goals of BICePs:

1. **Flexibility** (Organization is required)
  * Ability to add various types of experimental data and restraints whenever we please.
    
2. **Efficiency/Memory**
  * Power to run quickly, regardless of the amount of input data.
  * Parallelization on time consuming steps?
    - Operating System dependent (requires Linux or Unix in most cases)
    - > [scipy](https://scipy.github.io/old-wiki/pages/ParallelProgramming) - As the saying goes, **"premature optimization is the root of all evil"**. Using a multicore machine will provide at best a speedup by a factor of the number of cores available. Get your code working first, before even thinking about parallelization. Then ask yourself whether your code actually needs to be any faster. Don't embark on the bug-strewn path of parallelization unless you have to. 
  * Storing in memory only when necessary. 

3. **User-friendly**
  * sample_script.py should be easy to read/write
  * Users should be prompt with sufficient messages when program is running.
  * ```Help()``` should be informative - clear & concise.
  * Argparse 

------------------------------------------------------------------------------


## 1. **Flexibility**
I. **Reorganize code to contain:**
  * Class of restraints
  
### Quick Example:
Keep all Restraints in a single script...
```python

# filename = Restraint.py
# class for restraints 
class restraint(object):
    def __init__(self,data,...):
        self.data = data
        .
        .
        .
    def add_distance_restraint(self, i, j, exp_distance, model_distance=None,
                               equivalency_index=None):
        """Add an NOE NMR_Distance() object to the set"""

        # if the modeled distance is not specified, compute the distance from the conformation
        if model_distance == None:
            ri = self.conf.xyz[0,i,:]
            rj = self.conf.xyz[0,j,:]
            dr = rj-ri
            model_distance = np.dot(dr,dr)**0.5

        self.distance_restraints.append( NMR_Distance(i, j, model_distance, exp_distance,
                                                      equivalency_index=equivalency_index) )
        self.ndistances += 1
    
    def add_dihedral_restraint(self,...
    .
    .
    .
        
        
```



II. **Separate code:** Code that _points to_ specific methods from code that is executed when certain Exp. data is imported. 


Call on the Restraint Functions (methods) inside the Structure class.
**Look for "--->>" below**
As the data gets loaded in 

```python
from Restraint import *

# class for Structure that calls on restraint
class Structure(Restraint):
    def __init__(self,...):
        self.stuff = stuff
        .
        .
        .
    def load_expdata_noe(self, filename, verbose=False):
        """Load in the experimental NOE distance restraints from a .noe file format.
	"""

        # Read in the lines of the biceps data file
        b = RestraintFile_noe(filename=filename)
        data = []
        for line in b.lines:
		data.append( b.parse_line_noe(line) )  
        if verbose:
            print 'Loaded from', filename, ':'
            for entry in data:
                print entry
        ### distances ###
        # the equivalency indices for distances are in the first column of the *.biceps file
        equivalency_indices = [entry[0] for entry in data]
        if verbose:
            print 'distance equivalency_indices', equivalency_indices
        # add the distance restraints
        for entry in data:
            restraint_index, i, j, exp_distance = entry[0], entry[1], entry[4], entry[7]
            # Replacing self.add_dist... with the Restraint class
------>>    Restraint.add_distance_restraint(i, j, exp_distance, 
                                             model_distance=None,
                                             equivalency_index=None)
        # build groups of equivalency group indices, ambiguous group etc.
        self.build_groups_noe()

    .
    .
    .

# Eventually...        
if __name__ == '__main__':        
    if exp_data == filename.split('.')[-1]:   # if this = 'noe' (filename extension) then execute... 
        s = Structure('Gens/Gens%d.pdb'%i, lam*energies[i],
            expdata_filename_noe=expdata_filename_noe,
            expdata_filename_cs=expdata_filename_cs,
            use_log_normal_distances=False,dloggamma=np.log(1.01),
            gamma_min=0.2, gamma_max=5.0)


        s.load_expdata_noe(self, filename, verbose=False) # automatically adds restraint
    .
    .
    .



```

### Additional Notes:


```python
if __name__ == '__main__':
```

Using a module's __name__

Example: Using a module's __name__

				

Script:
```python
#!/usr/bin/python
# Filename: using_name.py
if __name__ == '__main__':
	print 'This program is being run by itself'
else:
	print 'I am being imported from another module'		
```


Output:
```python
$ python using_name.py
This program is being run by itself
```

```python
$ python
>>> import using_name
I am being imported from another module
>>>
```

#### [Click here for more info](http://ibiblio.org/g2swap/byteofpython/read/module-name.html)

------------------------------------------------------------------------------


## 2. **Efficiency/Memory** -- Let's not worry about this too much for now...

  * **Note that** parallel code is much more difficult to write/debug Vs. serial code (i.e., stepwise code). 

I think Using clever bulit in [python functions](https://docs.python.org/2/library/index.html) along with numpy as [scipy](https://scipy.github.io/old-wiki/pages/ParallelProgramming)  is the way to go.

#### Just a few definitions:
**Pipelining** - Breaking a task into steps performed by different processor units, with inputs streaming through, much like an assembly line; a type of parallel computing.

**Message Passing Interface (MPI)** - MPI primarily addresses the message-passing parallel programming model: data is moved from the address space of one process to that of another process through cooperative operations on each process.


------------------------------------------------------------------------------


